In [22]:
# !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

In [23]:
# !pip install "git+https://github.com/huggingface/transformers.git"

In [24]:
# !pip install trl

In [4]:
from unsloth import FastLanguageModel

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/mistral-7b-bnb-4bit',
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
    token=None,
    device_map='sequential',
    rope_scaling=None,
    fix_tokenizer=True,
    trust_remote_code=False,
    use_gradient_checkpointing=True)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [6]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments


In [7]:
max_seq_length = 2048

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls /content/drive/MyDrive/ACP/

communitech_jobs.csv  golden_jobs.csv  resume_data.csv	SFTtrainer.xlsx


In [117]:
import pandas as pd
data = pd.read_excel("/content/drive/MyDrive/ACP/SFTtrainer.xlsx", sheet_name="Sheet1")

In [118]:
data.head()

,Respond,Question
0,"Sure, I can help you with that.",what skills match with each of the jobs ?
1,"Sure, I can help you with that.",Can you provide insights on how my skills and ...
2,"Sure, I can help you with that.",Which companies in Toronto are currently hirin...
3,"Sure, I can help you with that.",Can you suggest any additional skills or certi...
4,"Sure, I can help you with that.",What are the typical salary ranges for the pos...


In [119]:
data['Question'] = data['Question'].astype(str)
data['Respond'] = data['Respond'].astype(str)

In [120]:
# !pip install datasets

In [122]:
from datasets import Dataset

# Assuming data['Question'] and data['Respond'] are lists
questions = data['Question']
responses = data['Respond']

# Creating the dataset
dataset = Dataset.from_dict({'Question': questions, 'Respond': responses})

In [123]:
print(dataset[0])

{'Question': 'what skills match with each of the jobs ?', 'Respond': 'Sure, I can help you with that.'}


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha=16,
    lora_dropout=0,
    bias='none',
    layers_to_transform=None,
    layers_pattern=None,
    use_gradient_checkpointing=True,
    random_state=3407,
    max_seq_length=2048,
    use_rslora=False,
    modules_to_save=None,
    init_lora_weights=True,
    loftq_config={},
)

In [101]:
# @title Default title text
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = None,
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

ValueError: You passed `packing=False` to the SFTTrainer, but you didn't pass a `dataset_text_field` or `formatting_func` argument.

In [97]:
inputs = tokenizer(
    [
        "Can you provide guidance on how to handle multiple job offers and make the best decision for my career goals?"
    ],
    return_tensors = "pt",
).to("cuda")

In [98]:
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [99]:

outputs

tensor([[    1,  2418,   368,  3084, 15988,   356,   910,   298,  4269,  5166,
          2389,  5751,   304,  1038,   272,  1489,  5161,   354,   586,  4835,
          7661, 28804,    13,    13,  5613, 28725,  7771, 28723,  4003,   460,
           741,  5944,   368,   541,  1388,   298,  1038,   272,  1489,  5161,
         28747,    13,    13, 28740, 28723,  3348,   409,  1430,  2405, 28742,
         28713, 19204, 28725,  7196, 28725,   304,  6057,  8812, 28723,    13,
         28750, 28723, 11772,   272,  2496,  5679,   304,   771, 28733, 10387,
          7873, 28723,    13, 28770, 28723, 24223, 11931,   272,  4723,   304,
           901,  1723,   727, 28723,    13, 28781, 28723,  3348,   409,   272,
          4628,   354,  4835,  8670,   466,   304,  5024,  4099, 28723,    13,
         28782, 28723,  1091,   950,  7478,   477,  4498,   734,   442, 15137,
           297,   574,  3681, 28723,    13, 28784, 28723,  8890,   727,   298,
          7967,   356,   574,  3327,   304,  5024,  

In [100]:
tokenizer.batch_decode(outputs)

["<s> Can you provide guidance on how to handle multiple job offers and make the best decision for my career goals?\n\nYes, absolutely. Here are some steps you can take to make the best decision:\n\n1. Assess each offer's compensation, benefits, and growth opportunities.\n2. Consider the company culture and work-life balance.\n3. Evaluate the location and commute time.\n4. Assess the potential for career advancement and professional development.\n5. Seek advice from mentors or colleagues in your network.\n6. Take time to reflect on your personal and professional goals.\n7. Negotiate salaries and benefits with each employer.\n8. Make a decision based on"]

In [73]:
model.save_pretrained("lora_model")

In [74]:
model.push_to_hub("sriku2412/unsloth_SFT_dotsLive")

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/sriku2412/unsloth_SFT_dotsLive
